In [4]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt

In [5]:
df_train = pd.read_csv('DailyDelhiClimateTrain.csv')
df_test = pd.read_csv('DailyDelhiClimateTest.csv')

In [6]:
df_train.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [32]:
dates = df_train['date'].values
temp = df_train['meantemp'].values

temp_test = df_test['meantemp'].values

# plt.figure(figsize=(15,5))
# plt.plot(dates, temp)
# plt.title('Temperature average', fontsize=20)
# plt.show()

In [36]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    print("hasil tf.expand_dims",series)

    ds = tf.data.Dataset.from_tensor_slices(series)
    print("hasil tf.data.Dataset.from_tensor_slices", ds.__str__())
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    print("hasil ds.window ",ds)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    print("hasil ds.flat_map ",ds)
    ds = ds.shuffle(shuffle_buffer)
    print("hasil ds.shuffle ",ds)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    print("hasil ds.map ", ds)
    return ds.batch(batch_size).prefetch(1) 

In [23]:
temp[:5]

array([10.        ,  7.4       ,  7.16666667,  8.66666667,  6.        ])

In [38]:
train_set = windowed_dataset(temp, window_size=60, batch_size=100, shuffle_buffer=1000)
test_set = windowed_dataset(temp_test, window_size=60, batch_size=100, shuffle_buffer=1000)

hasil tf.expand_dims tf.Tensor(
[[10.        ]
 [ 7.4       ]
 [ 7.16666667]
 ...
 [14.0952381 ]
 [15.05263158]
 [10.        ]], shape=(1462, 1), dtype=float64)
hasil tf.data.Dataset.from_tensor_slices <TensorSliceDataset shapes: (1,), types: tf.float64>
hasil ds.window  <WindowDataset shapes: DatasetSpec(TensorSpec(shape=(1,), dtype=tf.float64, name=None), TensorShape([])), types: DatasetSpec(TensorSpec(shape=(1,), dtype=tf.float64, name=None), TensorShape([]))>
hasil ds.flat_map  <FlatMapDataset shapes: (None, 1), types: tf.float64>
hasil ds.shuffle  <ShuffleDataset shapes: (None, 1), types: tf.float64>
hasil ds.map  <MapDataset shapes: ((None, 1), (None, 1)), types: (tf.float64, tf.float64)>
hasil tf.expand_dims tf.Tensor(
[[15.91304348]
 [18.5       ]
 [17.11111111]
 [18.7       ]
 [18.38888889]
 [19.31818182]
 [14.70833333]
 [15.68421053]
 [14.57142857]
 [12.11111111]
 [11.        ]
 [11.78947368]
 [13.23529412]
 [13.2       ]
 [16.43478261]
 [14.65      ]
 [11.72222222]
 [13.0416

In [40]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.LSTM(60),
    tf.keras.layers.Dense(30, activation = 'relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1),
])

In [42]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1.0000e-04, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
optimizer=optimizer,
metrics=['mae'])

history = model.fit(train_set, validation_data=test_set, epochs=50)

Epoch 1/50
15/15 [==============================] - 7s 153ms/step - loss: 24.4640 - mae: 24.9640 - val_loss: 25.6029 - val_mae: 26.1029
Epoch 2/50
15/15 [==============================] - 1s 77ms/step - loss: 24.4356 - mae: 24.9356 - val_loss: 25.5893 - val_mae: 26.0893
Epoch 3/50
15/15 [==============================] - 1s 85ms/step - loss: 24.7829 - mae: 25.2829 - val_loss: 25.5746 - val_mae: 26.0746
Epoch 4/50
15/15 [==============================] - 1s 82ms/step - loss: 24.7209 - mae: 25.2209 - val_loss: 25.5597 - val_mae: 26.0597
Epoch 5/50
15/15 [==============================] - 1s 83ms/step - loss: 24.5278 - mae: 25.0278 - val_loss: 25.5447 - val_mae: 26.0447
Epoch 6/50
15/15 [==============================] - 1s 79ms/step - loss: 24.5319 - mae: 25.0319 - val_loss: 25.5297 - val_mae: 26.0297
Epoch 7/50
15/15 [==============================] - 1s 79ms/step - loss: 24.3793 - mae: 24.8793 - val_loss: 25.5147 - val_mae: 26.0147
Epoch 8/50
15/15 [==============================] - 1s

In [ ]:
model.evaluate()

In [93]:
mat = tf.zeros([3,4]) # buat array tiga dimensi, di mana 
# dimensi 1 ada dua buah array, d 2 ada 3 buah array, d4 ada 4 buah array
mat_exp = tf.expand_dims(tf.zeros([3,4]), axis=-1) # mengexpand dimensi dari array input
# sebanyak satu dimensi, axis menentukan di mana dimensi tambahan itu ditambahkan
# axis=-1 menambah dimensi di dimensi terakhir
# axis = 0 menambah dimensi di dimensi awal
# axis = 1 menambah dimensi di dimensi kedua
x = [y for y in range(1,11)]
new_x = tf.expand_dims(x, axis=-1)
test = tf.data.Dataset.from_tensor_slices(new_x)
test = test.window(4 + 1, shift=1, drop_remainder=True)
test

<WindowDataset shapes: DatasetSpec(TensorSpec(shape=(1,), dtype=tf.int32, name=None), TensorShape([])), types: DatasetSpec(TensorSpec(shape=(1,), dtype=tf.int32, name=None), TensorShape([]))>

In [94]:
mat_exp

<tf.Tensor: shape=(3, 4, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.]]], dtype=float32)>

In [108]:
ds = tf.data.Dataset.from_tensor_slices(mat_exp)
ds = ds.window()
for window in ds:
    print(list(window.as_numpy_iterator()))

TypeError: window() missing 1 required positional argument: 'size'

In [117]:
dataset = tf.data.Dataset.range(7).window(3, shift=2,
                                        drop_remainder=True)
for window in dataset:
    print(list(window.as_numpy_iterator()))

[0, 1, 2]
[2, 3, 4]
[4, 5, 6]


In [43]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
data = np.array([[i] for i in range(50)])
targets = np.array([[i] for i in range(50)])
data_gen = TimeseriesGenerator(data, targets,
                               length=10, sampling_rate=2,
                               batch_size=2)
assert len(data_gen) == 20
batch_0 = data_gen[0]
x, y = batch_0
assert np.array_equal(x,
                      np.array([[[0], [2], [4], [6], [8]],
                                [[1], [3], [5], [7], [9]]]))
assert np.array_equal(y,
                      np.array([[10], [11]]))

TypeError: unsupported operand type(s) for *: 'int' and 'slice'